In [24]:
import numpy as np
import pandas as pd

import math
import extract

In [25]:
extract_players = extract.extract(2022091200, 109)

defense = extract.separate(extract_players)[0]
weapons = extract.separate(extract_players)[1]

In [26]:
print(defense)
print(weapons)

[               name position  player_x  player_y  player_a  player_s  \
1    Kareem Jackson       SS       NaN       NaN       NaN       NaN   
2               NaN      NaN     52.88     26.99      0.12      1.25   
3               NaN      NaN     53.01     27.03      0.13      1.27   
4               NaN      NaN     53.13     27.06      0.10      1.28   
5               NaN      NaN     53.25     27.10      0.08      1.28   
..              ...      ...       ...       ...       ...       ...   
204             NaN      NaN     64.40     32.31      3.87      1.62   
205             NaN      NaN     64.53     32.25      3.41      1.27   
206             NaN      NaN     64.64     32.19      2.81      1.03   
207             NaN      NaN     64.72     32.16      2.40      0.80   
208             NaN      NaN     64.79     32.14      2.02      0.62   

     player_dir  player_dis               outcome  
1           NaN         NaN                   NaN  
2         72.53        0.13  h

In [27]:
pos_defense = []
pos_weapon = []
for d in defense:
    pos_defense.append(d[['name', 'player_x', 'player_y']])
for w in weapons:
    pos_weapon.append(w[['name', 'player_x', 'player_y']])
    
pos_each_def = []
pos_each_weap = []

for pos in pos_defense:
    new_df = pd.DataFrame({'euc_position': [list(xy) for xy in zip(pos['player_x'], pos['player_y'])]})
    new_df['name'] = pos.at[1, 'name']
    new_df = new_df[['name', 'euc_position']]
    pos_each_def.append(new_df)
        
for pos in pos_weapon:
    new_df = pd.DataFrame({'euc_position': [list(xy) for xy in zip(pos['player_x'], pos['player_y'])]})
    new_df['name'] = pos.at[1, 'name']
    new_df = new_df[['name', 'euc_position']]
    pos_each_weap.append(new_df)

print(pos_each_def)
print(pos_each_weap)


[               name    euc_position
0    Kareem Jackson      [nan, nan]
1    Kareem Jackson  [52.88, 26.99]
2    Kareem Jackson  [53.01, 27.03]
3    Kareem Jackson  [53.13, 27.06]
4    Kareem Jackson   [53.25, 27.1]
..              ...             ...
203  Kareem Jackson   [64.4, 32.31]
204  Kareem Jackson  [64.53, 32.25]
205  Kareem Jackson  [64.64, 32.19]
206  Kareem Jackson  [64.72, 32.16]
207  Kareem Jackson  [64.79, 32.14]

[208 rows x 2 columns],                 name    euc_position
0    K'Waun Williams      [nan, nan]
1    K'Waun Williams  [50.78, 29.12]
2    K'Waun Williams   [50.88, 29.2]
3    K'Waun Williams  [50.97, 29.29]
4    K'Waun Williams  [51.05, 29.37]
..               ...             ...
203  K'Waun Williams   [58.09, 16.1]
204  K'Waun Williams  [58.46, 16.33]
205  K'Waun Williams  [58.81, 16.56]
206  K'Waun Williams  [59.16, 16.78]
207  K'Waun Williams   [59.5, 17.01]

[208 rows x 2 columns],              name    euc_position
0    Ronald Darby      [nan, nan]
1    

In [28]:
def proto_euc_distance(pos1, pos2):
    return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

In [29]:
num_weps = len(pos_each_weap)
defs_and_distances_total = []
for p_def in pos_each_def:
    pos_def_to_wep = []
    for w_def in pos_each_weap:
        combine = pd.DataFrame({
            'name_def': p_def['name'],
            'pos_def': p_def['euc_position'],
            'name_wep': w_def['name'],
            'pos_wep': w_def['euc_position']
        })
        combine['distance'] = combine.apply(lambda row: proto_euc_distance(row['pos_def'], row['pos_wep']), axis=1)
        pos_def_to_wep.append(combine)
    min_distances = []
    min_distances_weps = []
    for i in range(pos_def_to_wep[0].shape[0]):
        min_distance = 130**2 # some distance that is greater than the football field
        min_distance_wep = ""
        for k in range(num_weps):
            if pos_def_to_wep[k].at[i, 'distance'] < min_distance:
                min_distance = pos_def_to_wep[k].at[i, 'distance']
                min_distance_wep = pos_def_to_wep[k].at[i, 'name_wep'] 
        min_distances.append(min_distance)
        min_distances_weps.append(min_distance_wep)
    defs_and_distances = pd.DataFrame({
        'name_def': p_def['name'],
        'pos_def': p_def['euc_position'],
        'min_distance_wep': min_distances_weps,
        'min_distance': min_distances,
    })        
    defs_and_distances_total.append(defs_and_distances)

In [30]:
print(defs_and_distances_total)

[           name_def         pos_def min_distance_wep  min_distance
0    Kareem Jackson      [nan, nan]                   16900.000000
1    Kareem Jackson  [52.88, 26.99]        Noah Fant     11.606688
2    Kareem Jackson  [53.01, 27.03]        Noah Fant     11.661021
3    Kareem Jackson  [53.13, 27.06]        Noah Fant     11.706964
4    Kareem Jackson   [53.25, 27.1]        Noah Fant     11.746476
..              ...             ...              ...           ...
203  Kareem Jackson   [64.4, 32.31]    Tyler Lockett      1.060047
204  Kareem Jackson  [64.53, 32.25]    Tyler Lockett      1.051190
205  Kareem Jackson  [64.64, 32.19]    Tyler Lockett      1.057024
206  Kareem Jackson  [64.72, 32.16]    Tyler Lockett      1.090046
207  Kareem Jackson  [64.79, 32.14]    Tyler Lockett      1.103857

[208 rows x 4 columns],             name_def         pos_def min_distance_wep  min_distance
0    K'Waun Williams      [nan, nan]                   16900.000000
1    K'Waun Williams  [50.78, 29.1

In [31]:
for jitter in defs_and_distances_total:
    jitter.drop(index=jitter.index[0], axis=0, inplace=True)
    
print(defs_and_distances_total)

[           name_def         pos_def min_distance_wep  min_distance
1    Kareem Jackson  [52.88, 26.99]        Noah Fant     11.606688
2    Kareem Jackson  [53.01, 27.03]        Noah Fant     11.661021
3    Kareem Jackson  [53.13, 27.06]        Noah Fant     11.706964
4    Kareem Jackson   [53.25, 27.1]        Noah Fant     11.746476
5    Kareem Jackson  [53.38, 27.13]        Noah Fant     11.780514
..              ...             ...              ...           ...
203  Kareem Jackson   [64.4, 32.31]    Tyler Lockett      1.060047
204  Kareem Jackson  [64.53, 32.25]    Tyler Lockett      1.051190
205  Kareem Jackson  [64.64, 32.19]    Tyler Lockett      1.057024
206  Kareem Jackson  [64.72, 32.16]    Tyler Lockett      1.090046
207  Kareem Jackson  [64.79, 32.14]    Tyler Lockett      1.103857

[207 rows x 4 columns],             name_def         pos_def min_distance_wep  min_distance
1    K'Waun Williams  [50.78, 29.12]        Noah Fant      9.976999
2    K'Waun Williams   [50.88, 29.

In [32]:
print(defense)

[               name position  player_x  player_y  player_a  player_s  \
1    Kareem Jackson       SS       NaN       NaN       NaN       NaN   
2               NaN      NaN     52.88     26.99      0.12      1.25   
3               NaN      NaN     53.01     27.03      0.13      1.27   
4               NaN      NaN     53.13     27.06      0.10      1.28   
5               NaN      NaN     53.25     27.10      0.08      1.28   
..              ...      ...       ...       ...       ...       ...   
204             NaN      NaN     64.40     32.31      3.87      1.62   
205             NaN      NaN     64.53     32.25      3.41      1.27   
206             NaN      NaN     64.64     32.19      2.81      1.03   
207             NaN      NaN     64.72     32.16      2.40      0.80   
208             NaN      NaN     64.79     32.14      2.02      0.62   

     player_dir  player_dis               outcome  
1           NaN         NaN                   NaN  
2         72.53        0.13  h

In [33]:
for defender in defense:
    defender['name'] = defender.at[1, 'name']
    defender['position'] = defender.at[1, 'position']
    defender.drop(index=defender.index[0], axis=0, inplace=True)
    defender.reset_index(drop=True)
    
print(defense)

[               name position  player_x  player_y  player_a  player_s  \
2    Kareem Jackson       SS     52.88     26.99      0.12      1.25   
3    Kareem Jackson       SS     53.01     27.03      0.13      1.27   
4    Kareem Jackson       SS     53.13     27.06      0.10      1.28   
5    Kareem Jackson       SS     53.25     27.10      0.08      1.28   
6    Kareem Jackson       SS     53.38     27.13      0.13      1.31   
..              ...      ...       ...       ...       ...       ...   
204  Kareem Jackson       SS     64.40     32.31      3.87      1.62   
205  Kareem Jackson       SS     64.53     32.25      3.41      1.27   
206  Kareem Jackson       SS     64.64     32.19      2.81      1.03   
207  Kareem Jackson       SS     64.72     32.16      2.40      0.80   
208  Kareem Jackson       SS     64.79     32.14      2.02      0.62   

     player_dir  player_dis               outcome  
2         72.53        0.13  huddle_break_offense  
3         74.05        0.13   

In [34]:
for d in defense:
    d.reset_index(drop=True, inplace=True)
    d.index = d.index + 1
    
print(defense)

[               name position  player_x  player_y  player_a  player_s  \
1    Kareem Jackson       SS     52.88     26.99      0.12      1.25   
2    Kareem Jackson       SS     53.01     27.03      0.13      1.27   
3    Kareem Jackson       SS     53.13     27.06      0.10      1.28   
4    Kareem Jackson       SS     53.25     27.10      0.08      1.28   
5    Kareem Jackson       SS     53.38     27.13      0.13      1.31   
..              ...      ...       ...       ...       ...       ...   
203  Kareem Jackson       SS     64.40     32.31      3.87      1.62   
204  Kareem Jackson       SS     64.53     32.25      3.41      1.27   
205  Kareem Jackson       SS     64.64     32.19      2.81      1.03   
206  Kareem Jackson       SS     64.72     32.16      2.40      0.80   
207  Kareem Jackson       SS     64.79     32.14      2.02      0.62   

     player_dir  player_dis               outcome  
1         72.53        0.13  huddle_break_offense  
2         74.05        0.13   

In [39]:
merged_defenders = []
for idx, df in enumerate(defs_and_distances_total):
    df_t = defense[idx]
    df_t = df_t.merge(df, left_index=True, right_index=True)
    merged_defenders.append(df_t)
    
print(merged_defenders)

[               name position  player_x  player_y  player_a  player_s  \
1    Kareem Jackson       SS     52.88     26.99      0.12      1.25   
2    Kareem Jackson       SS     53.01     27.03      0.13      1.27   
3    Kareem Jackson       SS     53.13     27.06      0.10      1.28   
4    Kareem Jackson       SS     53.25     27.10      0.08      1.28   
5    Kareem Jackson       SS     53.38     27.13      0.13      1.31   
..              ...      ...       ...       ...       ...       ...   
203  Kareem Jackson       SS     64.40     32.31      3.87      1.62   
204  Kareem Jackson       SS     64.53     32.25      3.41      1.27   
205  Kareem Jackson       SS     64.64     32.19      2.81      1.03   
206  Kareem Jackson       SS     64.72     32.16      2.40      0.80   
207  Kareem Jackson       SS     64.79     32.14      2.02      0.62   

     player_dir  player_dis               outcome        name_def  \
1         72.53        0.13  huddle_break_offense  Kareem Jackson